In [ ]:
!pip install yfinance --upgrade --no-cache-dir
!pip install yahoo_fin --upgrade --no-cache-dir

In [ ]:
# DOLLAR COST AVERAGING
from datetime import date, timedelta, datetime

######## Select ticker, initial date, final date, initial amount, periodic amount to invest and periodicity in week days
ticker = "QQQ"
initial_date = date(1990,1,1)
#final_date = date.today()
final_date = date(2010,1,1)
initial_amount = 0
periodic_amount = 3000
week_days = 20
######## Select ticker, initial date, final date, initial amount, amount to purchases and periodicity
import numpy as np
from datetime import date, timedelta, datetime
import yfinance as yf  
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pandas as pd

DF0 = yf.download(ticker,initial_date,final_date)
DF0['Date'] = DF0.index
Date_Purchases=[]
Purchase_Prices=[]
Total_Shares = []
Total_Value = []

total_invested = 0
num_purchases = 0
j=-1
num_shares=0
for i in DF0.index:
  j+=1
  if (j%week_days == 0): 
    if (j==0): 
      num_shares += round((initial_amount + periodic_amount) /pd.to_numeric(DF0['Open'])[j],1)
      total_invested += round(initial_amount + periodic_amount,1)
    else: 
      num_shares += round( periodic_amount /pd.to_numeric(DF0['Open'])[j],1)
      total_invested += round(periodic_amount,1)
    Total_Shares.append(num_shares)
    Total_Value.append(round(DF0['Open'][j]*num_shares,1))

    num_purchases += 1
    Date_Purchases.append(DF0['Date'][j])
    Purchase_Prices.append(pd.to_numeric(DF0['Open'])[j])
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Candlestick(x=DF0['Date'],
                open=DF0['Open'],
                high=DF0['High'],
                low=DF0['Low'],
                close=DF0['Close']))
fig.update_layout(title_text="Dollar Cost Averaging for "+ticker,title_x=.4,title_y=.9)
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Price per Share for "+ticker)
fig.update_yaxes(title_text="Total Value", secondary_y=True)


fig.add_trace(go.Scatter(x=Date_Purchases, y=Purchase_Prices, name="Purchases",mode='markers',marker_symbol='circle',marker_color='black'))
fig.add_trace(go.Scatter(x=Date_Purchases, y=Total_Value, name="Total Value"),secondary_y=True)
fig.add_trace(go.Scatter(x=Date_Purchases, y=Total_Value, name="Total Value",mode='markers',marker_symbol='square',marker_color='black'),secondary_y=True)

fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()
final_value = round(DF0['Close'][-1]*Total_Shares[-1],1)
months =  (Date_Purchases[-1].year - Date_Purchases[0].year) * 12 + Date_Purchases[-1].month - Date_Purchases[0].month
print('Initial Date:',Date_Purchases[0].strftime('%B %d,%Y'), 'Final Date:',Date_Purchases[-1].strftime('%B %d,%Y'))
print("Number of Months = ",months,", Total Number of Purchases:", num_purchases)
rend = round(100*(final_value/total_invested-1),1)
print("Final Price for "+ticker+" = ",round(DF0['Close'][-1],1),", Final Number of Shares: ", round(Total_Shares[-1],1))
print("Total Invested:",round(total_invested,1),", Final Value:",final_value)
print("Return on Investment::", rend,"%")

[*********************100%***********************]  1 of 1 completed


Initial Date: March 10,1999 Final Date: December 30,2009
Number of Months =  129 , Total Number of Purchases: 137
Final Price for QQQ =  45.8 , Final Number of Shares:  10266.2
Total Invested: 411000 , Final Value: 469678.7
Return on Investment:: 14.3 %
